# ANSWER KEY: Debug Drill 04 - Overfitting Trap

**Bug:** LightGBM hyperparameters are too aggressive:
- `n_estimators=2000` (too many trees)
- `num_leaves=256` (too complex)
- `max_depth=-1` (unlimited depth)
- `min_child_samples=1` (can fit single examples)
- No early stopping

**Key Lesson:** Use regularization + early stopping. Large train/test gap = overfitting.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/shared/data/streamcart_customers.csv')

features = ['tenure_months', 'logins_last_30d', 'orders_last_30d', 
            'support_tickets_last_30d', 'nps_score']
X = df[features].fillna(0)
y = df['churn_30d']

## The Bug (Colleague's Code)

In [ ]:
# ===== BUGGY CODE =====
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_buggy = lgb.LGBMClassifier(
    n_estimators=2000,      # Too many!
    num_leaves=256,         # Too complex!
    max_depth=-1,           # No limit!
    min_child_samples=1,    # Can memorize!
    learning_rate=0.3,      # Too fast!
    random_state=42,
    verbose=-1
)

model_buggy.fit(X_train, y_train)

train_auc = roc_auc_score(y_train, model_buggy.predict_proba(X_train)[:, 1])
test_auc = roc_auc_score(y_test, model_buggy.predict_proba(X_test)[:, 1])

print(f"Buggy model:")
print(f"  Training AUC: {train_auc:.4f}")
print(f"  Test AUC: {test_auc:.4f}")
print(f"  Gap: {train_auc - test_auc:.4f}")
print(f"\nThis gap indicates severe overfitting!")

## Why This Is Wrong

| Parameter | Problem |
|-----------|--------|
| `n_estimators=2000` | Way too many trees without early stopping |
| `num_leaves=256` | Trees are too complex, can memorize data |
| `max_depth=-1` | No depth limit = arbitrarily deep trees |
| `min_child_samples=1` | Can create leaves with single examples |
| `learning_rate=0.3` | High LR + many trees = overfitting |

**The giveaway:** Train AUC near 1.0 but test AUC much lower = model memorized training data.

## The Fix

In [ ]:
# ===== FIXED CODE =====

# Split into train/val/test for early stopping
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

model_fixed = lgb.LGBMClassifier(
    n_estimators=1000,       # High, but early stopping will find optimal
    num_leaves=31,           # Reasonable complexity (default)
    max_depth=6,             # Limit depth
    min_child_samples=20,    # Require 20+ samples per leaf
    learning_rate=0.05,      # Lower = more gradual learning
    reg_alpha=0.1,           # L1 regularization
    reg_lambda=0.1,          # L2 regularization
    random_state=42,
    verbose=-1
)

# Early stopping monitors validation set
model_fixed.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
)

train_auc = roc_auc_score(y_train, model_fixed.predict_proba(X_train)[:, 1])
val_auc = roc_auc_score(y_val, model_fixed.predict_proba(X_val)[:, 1])
test_auc = roc_auc_score(y_test, model_fixed.predict_proba(X_test)[:, 1])

print(f"Fixed model:")
print(f"  Training AUC: {train_auc:.4f}")
print(f"  Validation AUC: {val_auc:.4f}")
print(f"  Test AUC: {test_auc:.4f}")
print(f"  Train-Test Gap: {train_auc - test_auc:.4f}")
print(f"\nTrees used: {model_fixed.best_iteration_} (stopped early!)")

In [ ]:
# Self-check
gap = abs(train_auc - test_auc)
assert gap < 0.10, f"Gap still too large: {gap:.4f}"
assert test_auc > 0.60, f"Test AUC too low: {test_auc:.4f}"
print("PASS: Overfitting controlled!")

## Parameter Comparison

| Parameter | Buggy | Fixed | Why |
|-----------|-------|-------|-----|
| n_estimators | 2000 | 1000 + early stopping | Let data decide |
| num_leaves | 256 | 31 | Simpler trees |
| max_depth | -1 (unlimited) | 6 | Limit complexity |
| min_child_samples | 1 | 20 | Require evidence |
| learning_rate | 0.3 | 0.05 | Slower, more stable |
| regularization | None | L1+L2 | Penalize complexity |

## Completed Postmortem

### Root cause:
- Hyperparameters allowed the model to memorize training data
- No early stopping meant all 2000 trees were used, even after validation performance peaked

### How we detected it:
- Massive gap between training AUC (~0.99) and test AUC (~0.60)
- Train performance "too good to be true"

### Prevention for next time:
- Always use early stopping with a validation set
- Start with conservative hyperparameters (default num_leaves=31, max_depth=6)
- Monitor train/val gap during tuning-if gap grows, add regularization